In [30]:
import ast

def node2state(node, info, level):
    info["level"] = level
    if isinstance(node, ast.AST):
        for name, val in ast.iter_fields(node):
            if "ast" in str(val):
                info[name] = None
            else:
                info[name] = val
        for name, val in ast.iter_fields(node):
            if name not in ('left', 'right'):
                node2state(val, info, level)
        info["type"] = node.__class__.__name__


def traverse(node, states, level=0):
    info = {}
    node2state(node, info, level)
    states.append(info)
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    traverse(item, states, level=level+1)
        elif isinstance(value, ast.AST):
            traverse(value, states, level=level+1)


def parse(code):
    return ast.parse(code)

def classify_state(states, characteristic, label):
    for state in states:
        if "label" not in state.keys():
            state["label"] = "Ok"
        for key, value in state.items():
            if key == characteristic or value == characteristic:
                state["label"] = label
                break


def code2states(code, critical_positions,labels):
    states = []
    tree = parse(code)
    traverse(tree, states)
    for index, critical in enumerate(critical_positions):
        classify_state(states, critical, labels[index])
    return states




In [32]:
# TESTING
#states = code2states("x = 20 + 30", [20,30],["bad","bad"])
#for state in states:
#    print(state)